# Analyzing Character Archetypes in TV Shows and Movies

This project involves fine-tuning a pre-trained language model to classify character descriptions into archetypes such as "hero," "sidekick," "mentor," and "villain." Using Hugging Face's Transformers and a custom dataset, the model is trained to predict the archetype of a character based on a brief description. The project showcases text classification using NLP and can be deployed for users to classify new character descriptions automatically.

### Prototype

In [10]:
from transformers import pipeline 

In [28]:
archetypes = ["Warrior", "Child", "Orphan", "Creator", "Caregiver", "Mentor", "Joker", "Magician", "Ruler", "Rebel", "Lover", "Seducer"]

classifier = pipeline("zero-shot-classification", model="facebook/bart-large-mnli")

def prototype_classify_character(character_description):
    result = classifier(character_description, candidate_labels=archetypes)
    return result['labels'][0]  # Return the highest scoring archetype

In [29]:
# Now try on an example
character_description =\
"""
Samwise Gamgee from The Lord of the Rings is a loyal, steadfast companion to Frodo Baggins. 
He begins as a humble gardener but proves to be one of the most courageous and selfless characters in the story. 
Despite being seemingly ordinary, Sam consistently demonstrates extraordinary bravery, especially when supporting 
Frodo on their perilous journey to destroy the One Ring. His deep sense of duty, unwavering friendship, and determination 
keep him by Frodo’s side, even in the darkest moments, making him a true hero in his own right, despite never seeking fame 
or glory.
"""

In [30]:
archetype = prototype_classify_character(character_description)
print(f"The character's archetype is: {archetype}")

The character's archetype is: Caregiver


### Dataset

In [31]:
import pandas as pd

In [32]:
arch_data = pd.read_csv("archetype_data.csv")